In [58]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re

import os


tf.random.set_seed(9999)

In [59]:
def get_dat_dir_path()->str:
    return os.path.abspath('../dat')
    
def get_chk_dir_path()->str:
    return os.path.abspath('../chk')

def get_train_csv_path()->str:
    dat_dir = get_dat_dir_path()
    # print(dat_dir)
    return os.path.join(dat_dir, "train.csv")

def get_test_csv_path()->str:
    dat_dir = get_dat_dir_path()
    return os.path.join(dat_dir, "test_x.csv")

def get_sample_sub_path()->str:
    dat_dir = get_dat_dir_path()
    return os.path.join(dat_dir, "sample_submission.csv")


In [60]:
train = pd.read_csv(get_train_csv_path())
test = pd.read_csv(get_test_csv_path())
sample_submission = pd.read_csv(get_sample_sub_path())

In [61]:
# preprocesscing

In [62]:
def alpha_num(txt:str)->str:
    return re.sub(r"[^A-Za-z0-9 ]", "", txt)

train["text"] = train["text"].apply(alpha_num)

In [63]:
train

,index,text,author
0,0,He was almost choking There was so much so muc...,3
1,1,Your sister asked for it I suppose,2
2,2,She was engaged one day as she walked in peru...,1
3,3,The captain was in the porch keeping himself c...,4
4,4,Have mercy gentlemen odin flung up his hands D...,3
...,...,...,...
54874,54874,Is that you Mr Smith odin whispered I hardly d...,2
54875,54875,I told my plan to the captain and between us w...,4
54876,54876,Your sincere wellwisher friend and sister LUC...,1
54877,54877,Then you wanted me to lend you money,3


In [64]:
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

def remove_stopwords(text:str) -> str:
    final_txt = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_txt.append(i.strip())
    return " ".join(final_txt)

In [65]:
train["text"] = train["text"].apply(alpha_num).apply(remove_stopwords)
test["text"] = test["text"].apply(alpha_num).apply(remove_stopwords)

In [66]:
train

,index,text,author
0,0,almost choking much much wanted say strange ex...,3
1,1,sister asked suppose,2
2,2,engaged one day walked perusing Janes last let...,1
3,3,captain porch keeping carefully way treacherou...,4
4,4,mercy gentlemen odin flung hands Dont write an...,3
...,...,...,...
54874,54874,Mr Smith odin whispered hardly dared hope come,2
54875,54875,told plan captain us settled details accomplis...,4
54876,54876,sincere wellwisher friend sister LUCY odin,1
54877,54877,wanted lend money,3


In [67]:
# x_train = train["text"].values

In [68]:
# x_train

In [69]:
x_train = np.array([x for x in train["text"]])
x_test = np.array([x for x in test["text"]])
y_train = np.array([x for x in train["author"]])

In [70]:
# x_train

In [71]:
# Modeling

In [72]:
vocab_size = 20000
embedding_dim = 128
max_length = 500
padding_type = "post"

In [73]:
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index

In [74]:
list(word_index)[:10]

['odin', 'not', 'said', 'no', 'one', 'mr', 'will', 'upon', 'now', 'man']

In [75]:
train_sequence = tokenizer.texts_to_sequences(x_train)
train_padded = pad_sequences(train_sequence, padding=padding_type, maxlen=max_length)

test_sequence = tokenizer.texts_to_sequences(x_test)
test_padded = pad_sequences(test_sequence, padding=padding_type, maxlen=max_length)

In [76]:
train_padded

array([[  141,  7259,    20, ...,     0,     0,     0],
       [  217,    58,   221, ...,     0,     0,     0],
       [  682,     5,    59, ...,     0,     0,     0],
       ...,
       [ 2407, 19738,   126, ...,     0,     0,     0],
       [  316,  3532,   164, ...,     0,     0,     0],
       [  203,     2,   811, ...,     0,     0,     0]], dtype=int32)

In [77]:
############################################################################
## this model part is from 텐서플로2와 머신러닝으로 시작하는 자연어처리 04. 텍스트분류
############################################################################

model_name = "cnn_classifier_en"
BATCH_SIZE=512
NUM_EPOCHS=10
VALID_SPLIT=0.1
MAX_LEN=train_padded.shape[1]

kargs = {
    "model_name": model_name,
    "vocab_size": vocab_size,
    "embedding_size": 256,
    "num_filters": 100,
    "dropout_rate": .5,
    "hidden_dimension": 250,
    "output_dimension": 5
}

In [78]:
class CNNClassifier(tf.keras.Model):
    def __init__(self, **kargs):
        super(CNNClassifier, self).__init__(name=kargs["model_name"])
        self.embedding = tf.keras.layers.Embedding(input_dim=kargs["vocab_size"],
                                                   output_dim=kargs["embedding_size"])
        self.conv_list = [ tf.keras.layers.Conv1D(filters=kargs["num_filters"],
                                                  kernel_size=kernel_size,   
                                                  padding="valid", 
                                                  activation="relu", 
                                                  kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.))
                            for kernel_size in [3,4,5,6] ]
        self.pooling = tf.keras.layers.GlobalMaxPooling1D()
        self.dropout = tf.keras.layers.Dropout(kargs["dropout_rate"])
        self.fc1 = tf.keras.layers.Dense(units=kargs["hidden_dimension"], activation="relu", 
                                         kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.)
                                         )
        self.fc2 = tf.keras.layers.Dense(units=kargs["output_dimension"], activation="softmax",
                                         kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.)
                                         )


    def call(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        x = tf.concat([self.pooling(conv(x)) for conv in self.conv_list], axis=-1)
        x = self.fc1(x)
        x = self.fc2(x)

        return x                                         


In [79]:
model = CNNClassifier(**kargs)


In [80]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])
model.build(train_padded.shape)
print(model.summary())            

Model: "cnn_classifier_en"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  5120000   
_________________________________________________________________
conv1d_6 (Conv1D)            multiple                  76900     
_________________________________________________________________
conv1d_7 (Conv1D)            multiple                  102500    
_________________________________________________________________
conv1d_8 (Conv1D)            multiple                  128100    
_________________________________________________________________
conv1d_9 (Conv1D)            multiple                  153700    
_________________________________________________________________
global_max_pooling1d_2 (Glob multiple                  0         
_________________________________________________________________
dropout_2 (Dropout)          multiple            

In [81]:
#num_epochs = 20
earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", 
                                                    min_delta=0.0001, 
                                                    patience=2)

checkpoint_path = os.path.join( get_chk_dir_path(), model_name, "weights.h5" )
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                monitor="val_accuracy", 
                                                verbose=1, 
                                                save_best_only=True, 
                                                save_weights_only=True)

if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir, exist_ok=True)

history = model.fit(train_padded, y_train, 
                    batch_size=BATCH_SIZE, 
                    epochs=NUM_EPOCHS, 
                    verbose=2, 
                    validation_split=VALID_SPLIT, 
                    callbacks=[earlystop_callback, cp_callback])

Epoch 1/10

Epoch 00001: val_accuracy improved from -inf to 0.65543, saving model to /Users/daewonyoon/github/daewonyoon/misc/2020/10_dacon_predict_novelist/chk/cnn_classifier_en/weights.h5
97/97 - 343s - loss: 1.3260 - accuracy: 0.4499 - val_loss: 0.9385 - val_accuracy: 0.6554
Epoch 2/10

Epoch 00002: val_accuracy improved from 0.65543 to 0.74016, saving model to /Users/daewonyoon/github/daewonyoon/misc/2020/10_dacon_predict_novelist/chk/cnn_classifier_en/weights.h5
97/97 - 346s - loss: 0.7299 - accuracy: 0.7329 - val_loss: 0.7021 - val_accuracy: 0.7402
Epoch 3/10

Epoch 00003: val_accuracy improved from 0.74016 to 0.74854, saving model to /Users/daewonyoon/github/daewonyoon/misc/2020/10_dacon_predict_novelist/chk/cnn_classifier_en/weights.h5
97/97 - 363s - loss: 0.5024 - accuracy: 0.8186 - val_loss: 0.6857 - val_accuracy: 0.7485
Epoch 4/10

Epoch 00004: val_accuracy did not improve from 0.74854
97/97 - 356s - loss: 0.3910 - accuracy: 0.8585 - val_loss: 0.7141 - val_accuracy: 0.7431
E

In [88]:
model.load_weights(checkpoint_path)

pred = model.predict(test_padded)

In [89]:
pred

array([[5.07397130e-02, 5.01002014e-01, 4.01831537e-01, 4.03336622e-02,
        6.09306945e-03],
       [1.31902099e-01, 7.42414355e-01, 5.64245041e-03, 1.66078806e-02,
        1.03433222e-01],
       [9.88556445e-01, 1.05835292e-02, 1.11594796e-04, 3.21773783e-04,
        4.26737737e-04],
       ...,
       [1.93125650e-03, 9.97475684e-01, 3.52293864e-05, 3.97617434e-04,
        1.60166877e-04],
       [2.68317647e-02, 9.68994319e-01, 1.06417213e-03, 2.38677370e-03,
        7.22960918e-04],
       [9.75115299e-01, 4.78760479e-03, 3.23811802e-03, 3.11765587e-03,
        1.37413004e-02]], dtype=float32)

In [90]:
pred.shape

(19617, 5)

In [91]:
test_padded.shape

(19617, 500)

In [92]:
sample_submission[[str(i) for i in range(5)]] = pred
sample_submission

,index,0,1,2,3,4
0,0,0.050740,0.501002,0.401832,0.040334,0.006093
1,1,0.131902,0.742414,0.005642,0.016608,0.103433
2,2,0.988556,0.010584,0.000112,0.000322,0.000427
3,3,0.113081,0.001731,0.724681,0.011298,0.149210
4,4,0.797600,0.023442,0.003550,0.112157,0.063251
...,...,...,...,...,...,...
19612,19612,0.001191,0.998573,0.000006,0.000138,0.000093
19613,19613,0.045008,0.009097,0.003445,0.000186,0.942264
19614,19614,0.001931,0.997476,0.000035,0.000398,0.000160
19615,19615,0.026832,0.968994,0.001064,0.002387,0.000723


In [93]:
sample_submission.to_csv("submission_cnn.csv", index=False, encoding="utf-8")